In [1]:
import numpy as np
import pylab as plt
import nfl_data_py as nfl

https://fivethirtyeight.com/methodology/how-our-nfl-predictions-work/

Adjustments to traditional Elo
- Home field advantage is worth 48 points plus an additional 4 points of Elo for every 1000 miles traveled
- A rest adjustment of 25 points of Elo whenever a team is coming off of a bye week
- Quarterback Adjustment that assigns every team and each individual QB a rolling performance rating, which can be used to adjust a team's "effective" Elo upward or downward in the event of a major injurt or other QB change.

## Elo Model

In [2]:
def predict_expected_score(R_A, R_B, base=400):
    """
    Returns the expected probability of a win given the Elo gramework
    """
    return (1 + 10 ** ((R_B - R_A) / base)) ** -1

def update_elo_score(score, expected, outcome, base=20, multiplier=1):
    """
    """
    return score + (outcome - expected) * base * multiplier

def margin_of_victory_multiplier(point_diff, winner_elo_diff):
    """
    """
    return np.log(point_diff + 1) * 2.2 * (winner_elo_diff * 0.001 + 2.2) ** -1

def update_all_scores(games, teams, home_advantage=48):
    """
    """
    for game in games:
        
        predicted_home = predict_expected_score(teams[game['home']] + home_advantage, teams[game['away']])
        predicted_away = predict_expected_score(teams[game['away']], teams[game['home']] + home_advantage)
        
        actual_home = game['outcome']
        actual_away = 1 - actual_home
        
        mov_mult = margin_of_victory_multiplier(game['point_diff'], np.abs(teams[game['home']] - teams[game['away']]))
        
        teams[game['home']] = update_elo_score(teams[game['home']], predicted_home, actual_home, multiplier=mov_mult)
        teams[game['away']] = update_elo_score(teams[game['away']], predicted_away, actual_away, multiplier=mov_mult)
        
    return teams

In [3]:
# Get all games from the 2022 NFL season
table = nfl.import_schedules([2022])

# Set a starting elo-score for all NFL teams
starting_elo = 1500
teams = {team_code: starting_elo for team_code in table["away_team"].unique()}

# Loop through each week
for week in range(1, 18):
    week_games = table[table["week"] == week]

    games = []
    for index, row in week_games.iterrows():
        if row["home_score"] > row["away_score"]:
            outcome = 1
        elif row["home_score"] < row["away_score"]:
            outcome = 0
        else:
            outcome = 0.5

        games.append(
            {
                "home": row["home_team"],
                "away": row["away_team"],
                "outcome": outcome,
                "point_diff": np.abs(row["home_score"] - row["away_score"]),
            }
        )

    teams = update_all_scores(games, teams)

In [4]:
teams

{'BUF': 1661.5539483682057,
 'NO': 1495.6351715896737,
 'CLE': 1508.311725901885,
 'SF': 1663.2595958963843,
 'PIT': 1516.3410891249193,
 'PHI': 1614.7146380243496,
 'IND': 1372.2084853648746,
 'NE': 1510.2990402355158,
 'BAL': 1553.1185193766808,
 'JAX': 1548.187670556704,
 'KC': 1643.7659541786695,
 'LV': 1482.5041346998185,
 'GB': 1522.7324400725922,
 'NYG': 1515.0716250773771,
 'TB': 1481.0412695222096,
 'DEN': 1375.3666741060276,
 'LAC': 1537.6925798178008,
 'MIA': 1472.042551768145,
 'NYJ': 1443.0240358907974,
 'WAS': 1459.1700377111472,
 'CAR': 1464.6704238207578,
 'ATL': 1426.3534774911589,
 'SEA': 1497.1904686235018,
 'CIN': 1638.065212746427,
 'HOU': 1340.2832771862036,
 'ARI': 1368.6594257958097,
 'CHI': 1352.5543874968514,
 'TEN': 1441.9365615167544,
 'MIN': 1543.6891717563399,
 'DET': 1523.1266576799926,
 'LA': 1382.2742438056525,
 'DAL': 1645.1555047967724}

In [5]:
dict(sorted(teams.items(), key=lambda item: item[1]))

{'HOU': 1340.2832771862036,
 'CHI': 1352.5543874968514,
 'ARI': 1368.6594257958097,
 'IND': 1372.2084853648746,
 'DEN': 1375.3666741060276,
 'LA': 1382.2742438056525,
 'ATL': 1426.3534774911589,
 'TEN': 1441.9365615167544,
 'NYJ': 1443.0240358907974,
 'WAS': 1459.1700377111472,
 'CAR': 1464.6704238207578,
 'MIA': 1472.042551768145,
 'TB': 1481.0412695222096,
 'LV': 1482.5041346998185,
 'NO': 1495.6351715896737,
 'SEA': 1497.1904686235018,
 'CLE': 1508.311725901885,
 'NE': 1510.2990402355158,
 'NYG': 1515.0716250773771,
 'PIT': 1516.3410891249193,
 'GB': 1522.7324400725922,
 'DET': 1523.1266576799926,
 'LAC': 1537.6925798178008,
 'MIN': 1543.6891717563399,
 'JAX': 1548.187670556704,
 'BAL': 1553.1185193766808,
 'PHI': 1614.7146380243496,
 'CIN': 1638.065212746427,
 'KC': 1643.7659541786695,
 'DAL': 1645.1555047967724,
 'BUF': 1661.5539483682057,
 'SF': 1663.2595958963843}